# Objectives 

- Familiarization with smart contarcts 
- Step through OPCODE level execution of a transaction
- Familiarize and set up web3 library and wallet APIs 
- Interacting with an ERC20 token


# Walkthrough smart contract

We have deployed smart contracts for the ERC20 tokens on the Goerli testnet. The contracts can be found on etherscan here: [LabUSD](https://goerli.etherscan.io/address/0x4966bb6cd9f3e042331b0798525b7970efb0d94a), [LabETH](https://goerli.etherscan.io/address/0xb85154e1948e52214a5f134172358fb5010f6282). Let us now go through the contract and describe what each ```function``` does.

### What does ```balanceOf``` and ```totalSupply``` do?

`balanceOf` is a view-only function that takes an address and returns the address's balance in the contract.

`totalSupply` returns is a view-only function that returns the total supply of tokens in the contract.

### What does ```transfer``` and ```transferFrom``` do ? Which of them needs you to run ```approve``` before it executes correctly ?

`transfer` takes a "to" address and an amount and moves that amount of tokens from the caller's address to the "to" address.

`transferFrom` takes a "from" and a "to" address and an amount and moves that amount of tokens from the "from" address to the "to" address, if the caller's address has the allowance to do so.

`transferFrom` requires an `approve` before it executes correctly to ensure that the caller's address has the allowance to spend the tokens of the "from" address.

### What events can be emitted by the contract ? [What is their use](https://consensys.net/blog/developers/guide-to-events-and-logs-in-ethereum-smart-contracts/) ?

The contract can emit the following events:

* Transfer of contract ownership to an address
* Transfer of tokens between two addresses
* Mint of tokens, increasing the total supply
* Burn of tokens from an address
* Approval of an address to spend some amount of the owner's tokens

Events are typically used for:

* Returning values from the smart contract to the UI
* Building async triggers with smart contract data, such as notifying an observer in the UI
* Storing data in a cheaper way on the Ethereum blockchain - there are APIs to work with data stored via emitted events

### How does the contract respond to airdrop requests ?


# Requesting tokens

To request the tokens we shall be using in the future for some in-class labs, we are going to use the ```web3``` library in python. The installation instructions can be found [here](https://web3py.readthedocs.io/en/v5/quickstart.html). We will view our transactions on the [Goerli testnet etherscan](https://goerli.etherscan.io/). 

We now send a transaction that uses this API to request some of the Lab ERC20 tokens from a [faucet contract](https://goerli.etherscan.io/address/0x4290913D2f785921A341995f66817Ec252D4eE13). The faucet allows each registered wallet to redeem a fixed number of tokens per day. The code used to do that has to be written below.

### Setup Web3 API
First, we connect your wallet to a Web3 API - this is a service that connects you to the Ethereum blockchain via a simple URL. 

To do that, first create a free account on [Infura](https://app.infura.io/register)(using any email ID).

Once you log in, you would see your Infura API key under the project name you had given. Use that key in the following code.

In [6]:
# Import web3
from web3 import Web3
from datetime import datetime, timedelta

w3 = Web3(Web3.HTTPProvider('https://goerli.infura.io/v3/41ae2ccc42d542c3a612a50df35ac6e2'))

print(w3.eth.block_number) #Check if the notebook is connected to the Infura node
print(w3.__dict__)

8445463
{'manager': <web3.manager.RequestManager object at 0x115916980>, 'codec': <eth_abi.codec.ABICodec object at 0x11563fac0>, 'eth': <web3.eth.Eth object at 0x115917940>, 'net': <web3.net.Net object at 0x1159176a0>, 'version': <web3.version.Version object at 0x115917850>, 'parity': <web3.parity.Parity object at 0x1159140a0>, 'geth': <web3.geth.Geth object at 0x115916e60>, 'testing': <web3.testing.Testing object at 0x115916440>, '_ens': <web3._utils.empty.Empty object at 0x11333a800>}


### Setup your ETH account details

In [7]:
# Wallet details

public_address = Web3.toChecksumAddress('0x5FA2389B85EF34aEFf561Aa8bfc3E9066eC6Ad2c') # see metamask for this
private_key = '28f325295825ac1453e8146ab92db58ccde7fd14076368130555b8d2d537bace' # see metamask for this

### Set up a contract instance
Now, we send a request transaction to the faucet, signed by your private key.

In [14]:
# Define the LabFaucet contract to connect to

contract_addr = Web3.toChecksumAddress('0x4290913D2f785921A341995f66817Ec252D4eE13')
abi_contract = '[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"address[]","name":"user_list","type":"address[]"},{"internalType":"address","name":"token_address","type":"address"}],"name":"deRegisterUsers","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"user_address","type":"address"},{"internalType":"address","name":"token_address","type":"address"}],"name":"getLatestRequestTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"user_address","type":"address"},{"internalType":"address","name":"token_address","type":"address"}],"name":"getRegistrationStatus","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"manager","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address[]","name":"user_list","type":"address[]"},{"internalType":"address","name":"token_address","type":"address"}],"name":"registerUsers","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract IERC20","name":"token","type":"address"}],"name":"requestTokens","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract IERC20","name":"token","type":"address"}],"name":"retrieveFaucetAmount","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract IERC20","name":"token","type":"address"}],"name":"retrieveFaucetBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"token_address","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"set_faucet_amount","outputs":[],"stateMutability":"nonpayable","type":"function"}]'
contract = w3.eth.contract(contract_addr, abi=abi_contract)

# Define the address of the ERC20 token you want

token_addr = Web3.toChecksumAddress('0xb85154E1948e52214A5F134172358Fb5010F6282')

### Create transaction data

In [15]:
### Create transaction data
txn = contract.functions.requestTokens(token_addr).build_transaction(
    {
        'nonce':w3.eth.getTransactionCount(public_address),
        'from':public_address,
    }
)

### Sign and post transaction
You should see the hash of your transaction printed.

In [13]:
# Post transaction - LabUSDC

signed_txn = w3.eth.account.signTransaction(txn, private_key) # Sign the txn with your private key 
txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction) # Send it onto the chain
print("Hash of swap transaction : ",w3.toHex(txn_hash)) # Search the txn hash on etherscan to see if it got confirmed

Hash of swap transaction :  0xd5a95241775bf1070b0a4cee26b530a2faed2777263fb6e29c0f0b3207c4e4f7


In [16]:
# Post transaction - LabETH

signed_txn = w3.eth.account.signTransaction(txn, private_key) # Sign the txn with your private key 
txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction) # Send it onto the chain
print("Hash of swap transaction : ",w3.toHex(txn_hash)) # Search the txn hash on etherscan to see if it got confirmed

Hash of swap transaction :  0x6a6f51a5347207c9b8ac25217df8f9d580b6aeba5b06ef607e3cb81f197bb1a1


## Alternate: Request tokens through Etherscan

Verified contracts on Etherscan allow you to interact with functions on the contract.
Open the LabFaucet contract by providing it's address. You can now access the contract's read and write functions.
Go to the ``write tab``, connect your metamask wallet and call the ``requestTokens`` function by providing your wallet's public address and the LabETH token's contract address. Confirm the transaciton on the metamask prompt.

# Add to MetaMask

- The token balances can now be tracked via MetaMask using the contract addresses of each token. 
- Use the "Import Tokens" link at the bottom of your MetaMask. 
- Enter the smart contract address for an ERC20 tokens we would be using in the class.
- Do this for each of the tokens - LabETH and LabUSD